# Importing Libraries

In [ ]:
import os
import numpy as np
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

# Uploading File And Creating Dataset

### Uploading File

In [ ]:
from google.colab import files
files.upload()

### Moving File

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

### Downloading Dataset

In [ ]:
import kagglehub
path = kagglehub.dataset_download("manmandes/malimg")
print("Path to dataset files:", path)

# Creating Model & Dataloader

### Creating Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(32 * 16 * 16, 128)
        self.fc2 = nn.Linear(128, 25)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

### Preaparing Dataset

In [ ]:
class MalimgDataset(Dataset):
    def __init__(self, root_dir):
        self.data = []
        self.labels = []
        self.encoder = LabelEncoder()

        classes = os.listdir(root_dir)
        all_labels = []
        for label in classes:
            folder = os.path.join(root_dir, label)
            for file in os.listdir(folder):
                img_path = os.path.join(folder, file)
                img = Image.open(img_path).convert('L').resize((64, 64))
                self.data.append(np.array(img))
                all_labels.append(label)

        self.labels = self.encoder.fit_transform(all_labels)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = torch.tensor(self.data[idx], dtype=torch.float32).unsqueeze(0) / 255.0
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return image, label

### Creating Dataloader

In [ ]:
batch_size = 32
learning_rate = 0.001

train_root_dir = os.path.join(path, 'malimg_dataset', 'train')
train_dataset = MalimgDataset(train_root_dir)
test_root_dir = os.path.join(path, 'malimg_dataset', 'test')
test_dataset = MalimgDataset(test_root_dir)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

model = SimpleCNN()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training & Test & Evaluation

### Training Model

In [ ]:
for epoch in range(10):
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}")

### Test Model & Evaluation

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")